In [1]:
!rm -rf ./logs

In [2]:
### run before slides

import os
import subprocess
import signal

class TensorBoardServer():
    def __init__(self):
        pass
    
    def start(self):
        self.process = subprocess.Popen("tensorboard --logdir ./logs --host 127.0.0.1 --port 6006",
                                  shell=True, preexec_fn=os.setsid)

    def stop(self):
        os.killpg(self.process.pid, signal.SIGTERM)
        
tb_server = TensorBoardServer()

# MXBoard Visualization Types

<center>![png](https://raw.githubusercontent.com/dmlc/web-data/master/mxnet/tensorboard/mxboard_cover.png)</center>

In [3]:
tb_server.start()

In [4]:
import numpy as np
from mxboard import SummaryWriter
import mxnet as mx

## 1) Graphs

### Graphs: Gluon Blocks

In [5]:
from mxnet.gluon import nn


net = nn.HybridSequential()
with net.name_scope():
    net.add(nn.Dense(128, activation='relu'))
    net.add(nn.Dense(64, activation='relu'))
    net.add(nn.Dense(10))

# The following three lines hybridize the network, initialize the parameters,
# and run forward pass once to generate a symbol which will be used later
# for plotting the network.
net.hybridize()
net.initialize()
net.forward(mx.nd.ones((1,)))

with SummaryWriter(logdir='./logs/types_example') as sw:
    sw.add_graph(net)

### Graphs: Gluon Blocks from Model Zoo

In [6]:
net = mx.gluon.model_zoo.vision.inception_v3()
net.hybridize()
net.initialize()
net.forward(mx.nd.ones((1, 3, 299, 299)))

with SummaryWriter(logdir='./logs/types_example') as sw:
    sw.add_graph(net)

### Graphs: Symbolic Graphs

In [7]:
data = mx.sym.Variable('data')
weight = mx.sym.Variable('weight')
bias = mx.sym.Variable('fc1_bias', lr_mult=1.0)
conv1 = mx.symbol.Convolution(data=data, weight=weight, name='conv1', num_filter=32, kernel=(3, 3))
conv2 = mx.symbol.Convolution(data=data, weight=weight, name='conv2', num_filter=32, kernel=(3, 3))
conv3 = conv1 + conv2
bn1 = mx.symbol.BatchNorm(data=conv3, name="bn1")
act1 = mx.symbol.Activation(data=bn1, name='relu1', act_type="relu")
sum1 = act1 + conv3
mp1 = mx.symbol.Pooling(data=sum1, name='mp1', kernel=(2, 2), stride=(2, 2), pool_type='max')
fc1 = mx.sym.FullyConnected(data=mp1, bias=bias, name='fc1', num_hidden=10, lr_mult=0)
fc2 = mx.sym.FullyConnected(data=fc1, name='fc2', num_hidden=10, wd_mult=0.5)
sc1 = mx.symbol.SliceChannel(data=fc2, num_outputs=10, name="slice_1", squeeze_axis=0)

with SummaryWriter(logdir='./logs/types_example') as sw:
    sw.add_graph(sc1)

## 2) Scalars

### Scalars: Single

In [8]:
x_vals = np.arange(start=0, stop=2 * np.pi, step=0.01)
y_vals = np.sin(x_vals)
with SummaryWriter(logdir='./logs/types_example') as sw:
    for x, y in zip(x_vals, y_vals):
        sw.add_scalar(tag='sin_curve', value=y, global_step=x * 100)

### Scalars: Multiple

In [10]:
import os

xs = np.arange(start=0, stop=2 * np.pi, step=0.01)
y_sin = np.sin(xs)
y_cos = np.cos(xs)
y_exp_sin = np.exp(y_sin)
y_exp_cos = np.exp(y_cos)
y_sin2 = y_sin * y_sin

with SummaryWriter(logdir='./logs/types_example') as sw:
    for x, y1, y2, y3, y4, y5 in zip(xs, y_sin, y_cos, y_exp_sin, y_exp_cos, y_sin2):
        sw.add_scalar('curves', {'sin': y1, 'cos': y2}, x * 100)  # log y1 with name 'sin' and y2 with name 'cos'
        sw.add_scalar('curves', ('exp(sin)', y3), x * 100)  # log y3 with name 'exp(sin)'
        sw.add_scalar('curves', ['exp(cos)', y4], x * 100)  # log y4 with name 'exp(cos)'
        sw.add_scalar('curves', y5, x * 100)  # log y5 without specifying scalar name
    if not os.path.exists('./exports'): os.makedirs('./exports')
    sw.export_scalars('./exports/scalars.json')

In [11]:
!python -m json.tool ./exports/scalars.json

{
    "./logs/types_example/curves/sin": [
        [
            1531959828.5916493,
            0.0,
            0.0
        ],
        [
            1531959828.595383,
            1.0,
            0.009999833334166664
        ],
        [
            1531959828.5956042,
            2.0,
            0.01999866669333308
        ],
        [
            1531959828.5957716,
            3.0,
            0.02999550020249566
        ],
        [
            1531959828.59594,
            4.0,
            0.03998933418663416
        ],
        [
            1531959828.596104,
            5.0,
            0.04997916927067833
        ],
        [
            1531959828.5962844,
            6.0,
            0.059964006479444595
        ],
        [
            1531959828.596482,
            7.000000000000001,
            0.06994284733753277
        ],
        [
            1531959828.5966408,
            8.0,
            0.0799146939691727
        ],
        [
            1531959828.596799,
    

            194.0,
            0.9326150140222005
        ],
        [
            1531959828.6539743,
            195.0,
            0.9289597150038693
        ],
        [
            1531959828.655696,
            196.0,
            0.9252115207881683
        ],
        [
            1531959828.65652,
            197.0,
            0.9213708061913954
        ],
        [
            1531959828.6567316,
            198.0,
            0.9174379552818098
        ],
        [
            1531959828.6568727,
            199.0,
            0.9134133613412252
        ],
        [
            1531959828.6570094,
            200.0,
            0.9092974268256817
        ],
        [
            1531959828.6571448,
            201.00000000000003,
            0.9050905633252009
        ],
        [
            1531959828.6573067,
            202.0,
            0.9007931915226273
        ],
        [
            1531959828.6574433,
            203.00000000000003,
            0.8964057411515598


        [
            1531959828.6244884,
            96.0,
            1.774502294373275
        ],
        [
            1531959828.6246233,
            97.0,
            1.7599748710935117
        ],
        [
            1531959828.624757,
            98.0,
            1.7454677071580194
        ],
        [
            1531959828.6248927,
            99.0,
            1.7309837016515186
        ],
        [
            1531959828.6250272,
            100.0,
            1.7165256995489035
        ],
        [
            1531959828.6259897,
            101.0,
            1.7020964911969167
        ],
        [
            1531959828.626128,
            102.0,
            1.687698811845016
        ],
        [
            1531959828.6262643,
            103.0,
            1.6733353412248129
        ],
        [
            1531959828.6265306,
            104.0,
            1.6590087031774259
        ],
        [
            1531959828.6282654,
            105.0,
            1.644721

## 3) Histograms

In [12]:
with SummaryWriter(logdir='./logs/types_example') as sw:
    for i in range(10):
        # create a normal distribution with fixed mean and decreasing std
        data = mx.nd.normal(loc=0, scale=10.0/(i+1), shape=(25, 4))
        sw.add_histogram(tag='norml_dist', values=data, bins=200, global_step=i)

## 4) Images

In [44]:
from scipy import misc

# convert from height*width*channel to channel*height*width, and values 0 to 1
face_img = mx.nd.array(misc.face()).transpose((2, 0, 1))/255
# duplicate image 15 times
face_batch = mx.nd.concatenate([face_img.expand_dims(0)] * 15, axis=0)

with SummaryWriter(logdir='./logs/types_example') as sw:
    # write batched faces to the event file
    sw.add_image(tag='faces', image=face_batch)

INFO:mxboard.event_file_writer:successfully opened events file: ./logs/types_example/events.out.tfevents.1530924190.ip-172-31-68-231
INFO:mxboard.event_file_writer:wrote 1 event to disk
INFO:mxboard.event_file_writer:wrote 1 event to disk


In [45]:
from IPython.display import HTML
def example_application():
    return HTML('<iframe width="1000" height="560" src="https://www.youtube.com/embed/4oMVMu6BKT8?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

In [46]:
example_application()

## 5) Embeddings

In [47]:
def transformer(data, label):
    data = data.reshape((-1,)).astype(np.float32)/255
    return data, label

batch_size = 2500
train_dataset = mx.gluon.data.vision.MNIST('./data', train=True, transform=transformer)
train_data = mx.gluon.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

for (data, label) in train_data:
    embedding = data
    labels = label
    images = data.reshape((batch_size, 1, 28, 28))
    break
    
with SummaryWriter(logdir='./logs/types_example') as sw:
    sw.add_embedding(tag='mnist', embedding=embedding, labels=labels, images=images)

INFO:mxboard.writer:saved embedding labels to ./logs/types_example/mnist
INFO:mxboard.writer:saved embedding images to ./logs/types_example/mnist
INFO:mxboard.writer:saved embedding data to ./logs/types_example/mnist


In [48]:
tb_server.stop()
tb_server.start()

## 6) Audio

In [49]:
frequency = 44100
# 44100 random samples between -1 and 1
data = mx.random.uniform(low=-1, high=1, shape=(frequency,))
max_abs_val = data.abs().max()
# rescale the data to the range [-1, 1]
data = data / max_abs_val
with SummaryWriter(logdir='./logs/types_example') as sw:
    sw.add_audio(tag='uniform_audio', audio=data, global_step=0)

INFO:mxboard.event_file_writer:successfully opened events file: ./logs/types_example/events.out.tfevents.1530924193.ip-172-31-68-231
INFO:mxboard.event_file_writer:wrote 1 event to disk
INFO:mxboard.event_file_writer:wrote 1 event to disk


## 7) Text

In [50]:
with SummaryWriter(logdir='./logs/types_example') as sw:
    # plain string
    greeting = 'Hello MXNet'
    sw.add_text(tag='simple_example', text=greeting)
    
    # markdown
    header_row = 'Hello | MXNet,\n'
    delimiter = '----- | -----\n'
    table_body = 'This | is\n' + 'so | awesome!'
    sw.add_text(tag='markdown_table', text=header_row+delimiter+table_body)

INFO:mxboard.event_file_writer:successfully opened events file: ./logs/types_example/events.out.tfevents.1530924276.ip-172-31-68-231
INFO:mxboard.event_file_writer:wrote 1 event to disk
INFO:mxboard.event_file_writer:wrote 1 event to disk
INFO:mxboard.event_file_writer:wrote 1 event to disk


## 8) PR Curve

In [51]:
with SummaryWriter(logdir='./logs/types_example') as sw:
    labels = mx.nd.uniform(low=0, high=2, shape=(100,), dtype=np.float32).astype(np.int32)
    predictions = mx.nd.uniform(low=0, high=1, shape=(100,), dtype=np.float32)
    sw.add_pr_curve(tag='pseudo_pr_curve', predictions=predictions, labels=labels, num_thresholds=120)

INFO:mxboard.event_file_writer:successfully opened events file: ./logs/types_example/events.out.tfevents.1530924918.ip-172-31-68-231
INFO:mxboard.event_file_writer:wrote 1 event to disk
INFO:mxboard.event_file_writer:wrote 1 event to disk


In [13]:
tb_server.stop()